In [1]:
# pip install langchain --upgrade
# Version: 0.0.149

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# The easy document loader for text
from langchain.document_loaders import PyPDFLoader

: 

### Load your data

In [3]:


loader = TextLoader(r'C:\Users\Ogidan\Desktop\hope\python3\Data_Science_projects\Deep learning\chatbots\langchain\dataset\breastfeeding.txt')
doc = loader.load()
print (f"You have {len(doc)} document")
print (f"You have {len(doc[0].page_content)} characters in that document")

You have 1 document
You have 34989 characters in that document


In [4]:
# data = loader.load()

In [5]:
# print (f'You have {len(data)} document(s) in your data')
# print (f'There are {len(data[0].page_content)} characters in your document')

### Chunk your data up into smaller documents

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(doc)

In [7]:
print (f'Now you have {len(texts)} documents')



Now you have 44 documents


### Create embeddings of your documents to get ready for semantic search

In [8]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

C:\Users\Ogidan\AppData\Local\Programs\Python\Python310\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [9]:
PINECONE_API_KEY = ''
PINECONE_API_ENV = '' 
OPENAI_API_KEY = "" 

In [10]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [11]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "breastfeeding" # put in the name of your pinecone index here

In [12]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

In [13]:
query = "how do i breastfeed?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [14]:
# Here's an example of the first document that was returned
docs[0].page_content[:250]

'Bottle Fed babies are rarely underfed, but it is important to feed on demand and not at the times you dictate. Check that the teat on the bottle is large enough for your babyâ€™s needs. If she cries or upset â€“ especially at the end of a feed, when '

### Query those docs to get your answer back

In [15]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [16]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [17]:
query = "how do i breastfeed?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [18]:
chain.run(input_documents=docs, question=query)

' Nursing more and skin-to-skin contact are both effective non-pharmacological means to help your baby feel better. However, if you are not able to breastfeed, you should consult with your healthcare provider to review your options and assess the situation.'